In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import copy

from pycbc import conversions, psd
from pycbc.filter import match

from simple_pe.param_est import metric
from pesummary.utils.samples_dict import SamplesDict
from pesummary.gw.conversions import convert

In [2]:
import logging
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)

## GW190412-like signal

## Read in PSDs, etc

In [3]:
ifos = ['H1', 'L1', 'V1']

psds = {'H1': 'aLIGOMidHighSensitivityP1200087',
        'L1': 'aLIGOMidHighSensitivityP1200087',
        'V1': 'AdVMidHighSensitivityP1200087',
        'f_low': 20.,
        'f_high': 8192,
        'length': 32
        }

psds['delta_f'] = 1. / psds['length']

approximant = 'IMRPhenomXPHM'

In [4]:
pycbc_psd = {}
for ifo in ifos:
    pycbc_psd[ifo] = psd.analytical.from_string(psds[ifo], psds['length'] * psds['f_high'] + 1, psds['delta_f'],
                                                psds['f_low'])

pycbc_psd['harm'] = 3. / sum([1. / pycbc_psd[ifo] for ifo in ifos])

/home/spxsf2/miniconda3/envs/igwn-py38/lib/python3.8/site-packages/pycbc/types/array.py:375: RuntimeWarning: divide by zero encountered in true_divide
  return self._data.__rtruediv__(other)


## Parameters

In [5]:
m1 = 30.
m2 = 10.
mc = conversions.mchirp_from_mass1_mass2(m1, m2)
eta = conversions.eta_from_mass1_mass2(m1, m2)
s1z = 0.3
s2z = 0.
chi_eff = conversions.chi_eff(m1, m2, s1z, s2z)
snr = 18

In [6]:
params = {'mass_1': m1,
          'mass_2': m2,
          'spin_1z': s1z,
          'spin_2z': s2z
        }

## Make a waveform

In [7]:
h0 = metric.make_waveform(params, params, 0., 1., psds['delta_f'], psds['f_low'], 
                         len(pycbc_psd['harm']), approximant)


Offset spins

In [8]:
start = copy.deepcopy(params)
start['spin_1z'] += 0.1
start['spin_2z'] += 0.1

In [9]:
dx_directions = ['spin_1z', 'spin_2z']
mismatch = 0.03

In [10]:
x_peak, match = metric.find_best_match(h0, start, dx_directions, psds['f_low'], pycbc_psd['harm'], 
                                     approximant=approximant, mismatch=mismatch)

In [11]:
while mismatch > 1e-3:
    x_peak, match = metric.find_best_match(h0, x_peak, dx_directions, psds['f_low'], pycbc_psd['harm'], 
                                     approximant=approximant, mismatch=mismatch, tolerance=0.05)
    print(match, mismatch)
    mismatch /=4


0.9956601333276608 0.03
0.9967401362177459 0.0075
0.9998035108839411 0.001875


In [12]:
print("best match: %.4f" % match)
print("at parameters")
print(x_peak)

best match: 0.9998
at parameters
{'mass_1': 30.0, 'mass_2': 10.0, 'spin_1z': 0.3462773308897064, 'spin_2z': -0.19460221262948763}


Offset masses

In [13]:
params = {'chirp_mass': mc,
          'symmetric_mass_ratio': eta,
          'chi_eff': chi_eff
        }
h0 = metric.make_waveform(params, params, 0., 1., psds['delta_f'], psds['f_low'], 
                         len(pycbc_psd['harm']), approximant)

In [14]:
start = copy.deepcopy(params)
start['chirp_mass'] += 0.1
start['chi_eff'] += 0.1

In [15]:
dx_directions = ['chirp_mass', 'symmetric_mass_ratio', 'chi_eff']
mismatch = 0.02

In [16]:
x_peak = start
while mismatch > 1e-3:
    x_peak, match = metric.find_best_match(h0, x_peak, dx_directions, psds['f_low'], pycbc_psd['harm'], 
                                     approximant=approximant, mismatch=mismatch, tolerance=0.05)
    print(match, mismatch)
    mismatch /=4

0.9978010218578469 0.02
0.9999552050735097 0.005
0.9999898235654362 0.00125


In [17]:
print("mismatch: %.4g" % (1 - match) )
print("at parameters")
print(x_peak)

mismatch: 1.018e-05
at parameters
{'chirp_mass': 14.647039106893113, 'symmetric_mass_ratio': 0.18932810940908154, 'chi_eff': 0.21830690143616632}


In [18]:
print(params)

{'chirp_mass': 14.650780257917608, 'symmetric_mass_ratio': 0.1875, 'chi_eff': 0.225}


Now with scipy

In [19]:
x_peak, match = metric.find_best_match(h0, start, dx_directions, psds['f_low'], pycbc_psd['harm'], 
                                     approximant=approximant, method='scipy')

In [29]:
print("mismatch: %.4g" % (1 - match) )
print("at parameters")
print(x_peak)

mismatch: 9.352e-06
at parameters
{'chirp_mass': 14.647224128477314, 'symmetric_mass_ratio': 0.18927511941008462, 'chi_eff': 0.21852659450854398}
